# factory-planning-ii.py

notebook 版本

## Factory Planning II Example
Source: http://www.gurobi.com/resources/examples/factory-planning-II

### version 2

区别在于不再指定维护计划, 
Instead of pre-defining a maintenance schedule for the machines, as was done in Factory Planning I, in this version of the model we will also optimize for the maintenance schedule.

维护计划修改为必须6个月维护一次
The maintenance requirements are as follows:
* Each machine must be down for maintenance in one month of the six.
* The exception to the above are the grinding machines where only two of them need to be down during the six months.

### Step 1: Import functions and create Constants

In [1]:
from gurobipy import *


In [2]:
# tested with Python 3.5.2 & Gurobi 7.0.1

products = ["Prod1", "Prod2", "Prod3", "Prod4", "Prod5", "Prod6", "Prod7"]
machines = ["grinder", "vertDrill", "horiDrill", "borer", "planer"]
time_periods = ["January", "February", "March", "April", "May", "June"]

# 获利表
profit_contribution = {"Prod1":10, "Prod2":6, "Prod3":8, "Prod4":4, "Prod5":11, "Prod6":9, "Prod7":3}

# 生产用时表(为何不以  Prod 为坐标)
time_table = {
    "grinder": {    "Prod1": 0.5, "Prod2": 0.7, "Prod5": 0.3,
                    "Prod6": 0.2, "Prod7": 0.5 },
    "vertDrill": {  "Prod1": 0.1, "Prod2": 0.2, "Prod4": 0.3,
                    "Prod6": 0.6 },
    "horiDrill": {  "Prod1": 0.2, "Prod3": 0.8, "Prod7": 0.6 },
    "borer": {      "Prod1": 0.05,"Prod2": 0.03,"Prod4": 0.07,
                    "Prod5": 0.1, "Prod7": 0.08 },
    "planer": {     "Prod3": 0.01,"Prod5": 0.05,"Prod7": 0.05 }
}


# number of machines down
# down = {("January","grinder"): 1, ("February", "horiDrill"): 2, ("March", "borer"): 1,
#         ("April", "vertDrill"): 1, ("May", "grinder"): 1, ("May", "vertDrill"): 1,
#         ("June", "planer"): 1, ("June", "horiDrill"): 1}
# change down to qMaintenance
qMaintenance = {"grinder":2, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1} # number of machines that need to be under maintenance

# number of each machine available
qMachine = {"grinder":4, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1} 


# market limitation of sells
upper = {
    ("January", "Prod1") : 500,
    ("January", "Prod2") : 1000,
    ("January", "Prod3") : 300,
    ("January", "Prod4") : 300,
    ("January", "Prod5") : 800,
    ("January", "Prod6") : 200,
    ("January", "Prod7") : 100,
    ("February", "Prod1") : 600,
    ("February", "Prod2") : 500,
    ("February", "Prod3") : 200,
    ("February", "Prod4") : 0,
    ("February", "Prod5") : 400,
    ("February", "Prod6") : 300,
    ("February", "Prod7") : 150,
    ("March", "Prod1") : 300,
    ("March", "Prod2") : 600,
    ("March", "Prod3") : 0,
    ("March", "Prod4") : 0,
    ("March", "Prod5") : 500,
    ("March", "Prod6") : 400,
    ("March", "Prod7") : 100,
    ("April", "Prod1") : 200,
    ("April", "Prod2") : 300,
    ("April", "Prod3") : 400,
    ("April", "Prod4") : 500,
    ("April", "Prod5") : 200,
    ("April", "Prod6") : 0,
    ("April", "Prod7") : 100,
    ("May", "Prod1") : 0,
    ("May", "Prod2") : 100,
    ("May", "Prod3") : 500,
    ("May", "Prod4") : 100,
    ("May", "Prod5") : 1000,
    ("May", "Prod6") : 300,
    ("May", "Prod7") : 0,
    ("June", "Prod1") : 500,
    ("June", "Prod2") : 500,
    ("June", "Prod3") : 100,
    ("June", "Prod4") : 300,
    ("June", "Prod5") : 1100,
    ("June", "Prod6") : 500,
    ("June", "Prod7") : 60,
}



### Step 2: Create  model

In [3]:
storeCost = 0.5
storeCapacity = 100
endStock = 50
hoursPerMonth = 2*8*24

model = Model('Factory Planning I')



### Step 3: Create activitiy variables

In [4]:
manu = model.addVars(time_periods, products, vtype=GRB.CONTINUOUS, name="Manu")
held = model.addVars(time_periods, products , ub=storeCapacity, vtype=GRB.CONTINUOUS, name="held")
sell = model.addVars(time_periods, products, ub=upper, name="Sell") # quantity

down_vars = model.addVars(time_periods, machines, vtype=GRB.INTEGER, name="down_vars")

### Step 4: Set objective function

In [5]:
# Objective
obj = quicksum(profit_contribution[product] * sell[time_period, product] -  storeCost * held[time_period, product]  for time_period in time_periods    
                    for product in products)

model.setObjective(obj, GRB.MAXIMIZE)

### Step 5: Add constraints

In [6]:
## 3 个库存条件; 条件2采用了类似差分方程的写法
# At the start of January there is no product inventory
# Init + Manu = Sell + Hold 
# 注意:如果使用错 addConstr 而不是 addConstr[s]
# 会报错误: TypeError: unsupported operand type(s) for -: 'generator' and 'NoneType'
#  或是 NameError: name 'p' is not defined
# init month
model.addConstrs( (manu[time_periods[0],p] == sell[time_periods[0],p] 
                  + held[time_periods[0],p] for p in products), name="held_init_Jan_")

# Hold[-1] + Manu = Sell + Hold 
model.addConstrs( (held[time_periods[time_periods.index(t)-1],p] + manu[t,p] == sell[t,p] 
                  + held[t,p] for p in products for t in time_periods if t != time_periods[0]), name="held_init_Jan_")

# last month:  by the end of June there should be 50 units of each product in inventory.
model.addConstrs( (held[time_periods[-1],p] == endStock for p in products), name="held_last_month_")

## 两个设备时间约束; 采用了 线性相乘取和的写法  
# time_table[m] = {    "Prod1": 0.5, "Prod2": 0.7, "Prod5": 0.3, "Prod6": 0.2, "Prod7": 0.5 }
# manu[t,p] = "Prod1": 20 , ...
# quicksum( manu[t,p] * time_table[m][p] )
# 设备 down 的时间约束
# # Capacity

# -ii version 将  down 修改为 down_vars
model.addConstrs(  (quicksum( manu[t,p] * time_table[m][p] for p in time_table[m] )
                    <= (qMachine[m] -  down_vars[t,m] ) * hoursPerMonth for t in time_periods for m in machines 
                             ), name="machine_has_down") 


# model.addConstrs( (down_vars.sum('*', m) == qMaintenance[m]  for m in machines ), "Maintenance")
# 这两种写法效果一样
model.addConstrs( down_vars.sum('*', m) == qMaintenance[m]  for m in machines )



# 设备 不 down 的时间约束: (这个约束条件是可以不加的, 最优值一样)
# 比 for p in products if time_table[m].has_key(p))
# 更好的写法: for product in time_table[m]
# model.addConstrs(  (quicksum(( manu[t,p] * time_table[m][p] for p in products if time_table[m].has_key(p)))
#                     <= qMachine[m] * hoursPerMonth for t in time_periods for m in machines 
#                     if (t,m) not in down), name="machine_not_down") 

{'borer': <gurobi.Constr *Awaiting Model Update*>,
 'grinder': <gurobi.Constr *Awaiting Model Update*>,
 'horiDrill': <gurobi.Constr *Awaiting Model Update*>,
 'planer': <gurobi.Constr *Awaiting Model Update*>,
 'vertDrill': <gurobi.Constr *Awaiting Model Update*>}

### Step 6:  Add Our Constrains & Solve model

In [7]:
#model.addConstrs (  name="down")

model.optimize()

print "无任何机器下线最佳值为: obj = 116455"
print "有机器下线的最佳值应为: # obj = Best objective 1.088550000000e+05"
if int(model.ObjVal) == int(1.088550000000e+05):
    print "\n最佳值 {} 与答案一致.".format(model.ObjVal)
else:
    print "\n而不是 {} ....".format(model.ObjVal)


Optimize a model with 84 rows, 156 columns and 348 nonzeros
Variable types: 126 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-02, 4e+02]
  Objective range  [5e-01, 1e+01]
  Bounds range     [6e+01, 1e+03]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective -175
Presolve removed 22 rows and 27 columns
Presolve time: 0.00s
Presolved: 62 rows, 129 columns, 278 nonzeros
Variable types: 105 continuous, 24 integer (12 binary)

Root relaxation: objective 1.164550e+05, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 116455.000    0   13 -175.00000 116455.000      -     -    0s
H    0     0                    92755.000000 116455.000  25.6%     -    0s
H    0     0                    95895.000000 116455.000  21.4%     -    0s
H    0     0                    101825.00000 116455.000  14.4%     -    0s
 

### Step 8: Print variable values for optimal solution

In [8]:
# Display solution (print the name of each variable and the solution value)

def printAllVars():
    """
    打印所有变量
    """
    for v in model.getVars():
        if v.X != 0:
            print("%s %f" % (v.Varname, v.X))
# printAllVars()
print "\tB:  ",
for w in time_periods:
    print " [{0}]".format(w),
print 
for p in products:
    print "\t[A_{0}]".format(p), 
    # print('Plant %s open' % p)
    for w in time_periods:
        #if transport[w,p].x > 0:
        #    print('  Transport %g units to warehouse %s' % \
        #          (transport[w,p].x, w))
        print "\t{}".format(int(manu[w, p].x)), 
    print
        


	B:    [January]  [February]  [March]  [April]  [May]  [June]
	[A_Prod1] 	500 	600 	400 	0 	0 	550
	[A_Prod2] 	1000 	500 	700 	0 	100 	550
	[A_Prod3] 	300 	200 	100 	0 	500 	150
	[A_Prod4] 	300 	0 	100 	0 	100 	350
	[A_Prod5] 	800 	400 	600 	0 	1000 	1150
	[A_Prod6] 	200 	300 	400 	0 	300 	550
	[A_Prod7] 	100 	150 	200 	0 	0 	110
